<a href="https://colab.research.google.com/github/MyDearGreatTeacher/AI4high/blob/master/Keras_RNN_LSTM_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU 版本

2090504

## 資料來源

https://www.dlology.com/blog/how-to-train-keras-model-x20-times-faster-with-tpu-for-free/


https://colab.research.google.com/drive/1QZf1WeX3EQqBLeFeT4utFKBqq-ogG1FN#scrollTo=CB43mV-TD1vb&forceEdit=true&offline=true&sandboxMode=true

#  Tutorial - [How to train Keras model x20 times faster with TPU for free](https://www.dlology.com/blog/how-to-train-keras-model-x20-times-faster-with-tpu-for-free/)

In [3]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 3.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3


In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding

In [0]:

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words (among top max_features most common words)
maxlen = 500

# Load data
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Reverse sequences
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

# Pad sequences
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [0]:
def make_model(batch_size=None):
  source = Input(shape=(maxlen,), batch_size=batch_size, dtype=tf.int32, name='Input')
  embedding = Embedding(input_dim=max_features, output_dim=128, name='Embedding')(source)
  # lstm = Bidirectional(LSTM(32, name = 'LSTM'), name='Bidirectional')(embedding)
  lstm = LSTM(32, name = 'LSTM')(embedding)
  predicted_var = Dense(1, activation='sigmoid', name='Output')(lstm)
  model = tf.keras.Model(inputs=[source], outputs=[predicted_var])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='binary_crossentropy',
      metrics=['acc'])
  return model
   

In [4]:
tf.keras.backend.clear_session()
training_model = make_model(batch_size = 128)
training_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (128, 500)                0         
_________________________________________________________________
Embedding (Embedding)        (128, 500, 128)           1280000   
_________________________________________________________________
LSTM (LSTM)                  (128, 32)                 20608     
_________________________________________________________________
Output (Dense)               (128, 1)                  33        
Total params: 1,300,641
Trainable params: 1,300,641
Non-trainable params: 0
_________________________________________________________________


In [5]:
import os
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.summary()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.2.243.202:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 15886053908250569762)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15004378336908547387)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4384857135564577052)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:

In [6]:
import time
start_time = time.time()

history = tpu_model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128 * 8,
                    validation_split=0.2)
tpu_model.save_weights('./tpu_model.h5', overwrite=True)

print("--- %s seconds ---" % (time.time() - start_time))


Train on 25000 samples, validate on 5000 samples
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(128,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(128, 500), dtype=tf.int32, name='Input_10'), TensorSpec(shape=(128, 1), dtype=tf.float32, name='Output_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 3.424459934234619 secs
INFO:tensorflow:Setting weights on TPU model.
23552/25000 [===========================>..] - ETA: 0s - loss: 0.6933 - acc: 0.4856INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(53,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(53, 500), dtype=tf.int32, name='Input_10'), TensorSpec(shape=(53, 1), dtype=tf.float32,

In [0]:
# history = tpu_model.fit(x_train, y_train,
#                     epochs=20,
#                     batch_size=128 * 8,
#                     validation_split=0.2)
# tpu_model.save_weights('./tpu_model.h5', overwrite=True)

In [8]:
inferencing_model = make_model(batch_size=None)
inferencing_model.load_weights('./tpu_model.h5')
inferencing_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 500)               0         
_________________________________________________________________
Embedding (Embedding)        (None, 500, 128)          1280000   
_________________________________________________________________
LSTM (LSTM)                  (None, 32)                20608     
_________________________________________________________________
Output (Dense)               (None, 1)                 33        
Total params: 1,300,641
Trainable params: 1,300,641
Non-trainable params: 0
_________________________________________________________________


In [9]:
inferencing_model.evaluate(x_test, y_test)

25000/25000 [==============================] - 65s 3ms/sample - loss: 0.6107 - acc: 0.8267


[0.6106908162593842, 0.82672]

In [11]:
#tpu_model.evaluate(x_test, y_test, batch_size=128 * 8)
tpu_model.evaluate(x_test, y_test, batch_size=128)

INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id_10'), TensorSpec(shape=(16, 500), dtype=tf.int32, name='Input_10'), TensorSpec(shape=(16, 1), dtype=tf.float32, name='Output_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 12.048400163650513 secs
24704/25000 [============================>.] - ETA: 0s - loss: 0.6112 - acc: 0.8266INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(5,), dtype=tf.int32, name='core_id_10'), TensorSpec(shape=(5, 500), dtype=tf.int32, name='Input_10'), TensorSpec(shape=(5, 1), dtype=tf.float32, name='Output_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed

[0.6106106699752808, 0.82668]

In [13]:
#tpu_model.evaluate(x_test, y_test, batch_size=128 * 8)
tpu_model.evaluate(x_test, y_test, batch_size=128)

25000/25000 [==============================] - 4s 155us/sample - loss: 0.6106 - acc: 0.8267


[0.6106106699752808, 0.82668]

In [14]:
inferencing_model.predict(x_test[:10])> 0.5

array([[False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True]])

In [15]:
y_test[:10]

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

In [16]:
tpu_model.predict_on_batch(x_train[:128 * 8])>0.5

INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(128, 500), dtype=tf.int32, name='Input_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 18.751532554626465 secs


array([[ True],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

## Download the trained model weights to your local file system

In [0]:
from google.colab import files

files.download('./tpu_model.h5')